# IWGGMS 21 - June 9th-12th, Takamtasu, Japan

Welcome to the interactive demonstration of our new retrieval toolkit, created by the University of Maryland and NASA Goddard Space Flight Center! In this demonstration, we will run an application to retrieve XCO$_2$ from an OCO-2 measurement. The application itself is an independent implementation of NASA's ACOS algorithm.

**Purpose of this demonstration**


<div class="alert alert-block alert-warning">
<b>Note regarding performance</b> When aerosols are retrieved in this demonstration set-up, an iteration with all three bands takes about one minute each. 
</div>

**Acknowledgements and References**

Spectroscopy tables were provided by NASA JPL

## Set-up and choosing configuration

In [ ]:
# Point us to the location of the XRTM RT library
if !("XRTM_PATH" in ENV)
    ENV["XRTM_PATH"] = "~/xrtm"
end

# Let us observe the progress during the RT computations, set this to "0" if you don't want progress bars
ENV["XRTM_PROGRESS"] = "1"


# For good font rendering, we suggest installing the "JuliaMono" font
# which provides full support for the various Unicode glyphs.
#ENV["GKS_FONTPATH"] = "~/Library/Fonts/"

In [ ]:
# Load these modules ahead of running the application
using Plots, LaTeXStrings
gr()

In [ ]:
Plots.default()

Plots.default(
    fontfamily = "JuliaMono-Regular",
    titlefont = (10, "JuliaMono-Regular"),
    legendfont = (8, "JuliaMono-Regular"),
    guidefont = (8, "JuliaMono-Regular", :black),
    tickfont = (8, "JuliaMono-Regular", :black),
    rightmargin=5Plots.mm,
    leftmargin=5Plots.mm,
    bottommargin=5Plots.mm
)

## Run the retrieval

Below is the cell which runs the retrieval for an example scene, a real measurement from NASAs OCO-2. Feel free to experiment with any of these major settings. You can easily change the 3-band retrieval to a 2-band retrieval by changing the follwing line

    "--spec", "1,2,3",

to

    "--spec", "1,2",

which would only retrieve the O$_2$ A-band (1) and the Weak CO$_2$ band (2), omitting the Strong CO$_2$ band (3). Or you can do a single-band retrieval by just writing only one of the three numbers:

    "--spec", "3",

Note that the demonstration algorithm adjust the retrival state vector according to the band choices. For example, surface pressure is only retrieved when the O$_2$ A-band (1) is retrieved. Similarly, the CO$_2$ profile is only retrieved when either band 2 or 3 are included (or both).

Further, you can switch off the aerosol retrieval by changing

    "--aerosols", "true",

to 

    "--aerosols", "false",



**Run the retrieval!**

The cell below will execute the retrieval according to the set-up that is provided through the arguments in the list `my_args`. Some amount of output will follow, depending on how many iterations are performed etc.

In [ ]:
# Define command-line arguments

my_args = [
    "--solar_model",  "./example_data/l2_solar_model.h5", # Path to the solar model file
    "--L1b", "./example_data/2021030111564431_inputs.h5", # Path to the L1B data location
    "--L2Met", "./example_data/2021030111564431_inputs.h5", # Path to the L2Met data location
    "--L2CPr", "./example_data/2021030111564431_inputs.h5", # Path to the L2CPr data location
    #######################################################
    "--sounding_id", "2021030111564431", # Sounding ID to retrieve
    "--spec", "1,2,3", # Which spectra to retrieve? 1 = O2-A, 2 = Weak CO2 (1.6 µm), 3 = Strong CO2 (2.06 µm)
    "--aerosols", "true", # Co-retrieve aerosol parameters? (height, width, AOD)
    "--o2_scale", "1.0048", # Spectroscopy scaling factor for Oxygen
    "--co2_scale_weak", "0.994", # Spectroscopy scaling factor for CO2 for the weak CO2 band
    "--co2_scale_strong", "0.998", # Spectroscopy scaling factor for CO2 for the strong CO2 band
    "--gamma", "1000.0", # Levenberg-Marquardt γ parameter (to avoid diverging steps, we make this large)
    "--dsigma_scale", "5.0", # dσ^2 ~ a parameter that controls convergence, make this bigger for faster convergence
    "--max_iterations", "10", # Number of maximal iterations
]

# Get rid of existing command line arguments
empty!(ARGS)

# Push them in
for a in my_args
    push!(ARGS, a)
end

# Run the retrieval and return the Buffer and Solver objects! 
# (Note! This will take several minutes)
@time buf, solver = include("acos-goddard/run.jl");

**FINISHED**

Once the above cell finishes computing, the retrieval was hopefully successful! From here on, we will examine the results and inspect some of the inputs that enter the retrieval algorithm.

## Plot the multi-band fit as the inversion sees it

The most obvious place to start analyzing the retrieval is to look at the spectral fit. Below we can plot the radiances measured by OCO-3 (in blue) and then overlay the last forward model computation that the retrieval produced.

In [ ]:
# This grabs the measured and modeled radiances
measured = RE.get_measured(solver);
modelled = RE.get_modeled(solver);

# Plot the multi-band fit
Plots.plot(measured, size=(1100, 500), linewidth=2, label="Measured", leftmargin=5Plots.mm, rightmargin=5Plots.mm);
Plots.plot!(modelled, linestyle=:dash, label="Fit")
Plots.xlabel!("Spectral sample #");
Plots.ylabel!("Radiance\n[ph μm⁻¹ m⁻² s⁻¹ sr⁻¹]");
Plots.title!("Full fit as seen by inversion algebra")

## Closer look at the individual spectral windows and their spectral residuals

In [ ]:
chi2 = RE.calculate_chi2(solver);


# Loop through each spectral window
for swin in buf.spectral_window

    # 
    rt = buf.rt[swin];

    # For this spectral window, grab the wavelenght, measured radiance, model radiance,
    # and noise-equivalent radiances. These will the vectors of the same length that can
    # be used for plotting.
    wavelength = RE.get_wavelength(solver, swin);
    measured = RE.get_measured(solver, swin);
    modeled = RE.get_modeled(solver, swin);
    noise = RE.get_noise(solver, swin);

    # We calculate the relative residuals
    resid = @. (modeled - measured) / noise;

    # Create a two-panel plot
    
    # First plot - measured and modeled radiance
    p1 = Plots.plot(wavelength, measured, label="Measured", linewidth=2,
        leftmargin=5Plots.mm, rightmargin=5Plots.mm, bottommargin=5Plots.mm, size=(800, 500));
    Plots.plot!(wavelength, modeled, label="Fit", linestyle=:dash)
    Plots.title!(@sprintf "χ2 = %.2f" chi2[swin])
    Plots.ylabel!("Radiance\n[$(rt.radiance_unit)]")
    
    # Second plot - radiance residuals
    p2 = Plots.plot(wavelength, resid, label="Residual", linewidth=2,
        leftmargin=5Plots.mm, rightmargin=5Plots.mm, bottommargin=5Plots.mm);
    Plots.xlabel!("Wavelength [$(swin.ww_unit)]")
    Plots.ylabel!("Residual\n(fraction of noise)")
    
    
    disp_plot = plot(p1, p2, layout=(2, 1))
    display(disp_plot)
    
end

## Posterior analysis

We can look at the atmospheric state in terms of the Xgas, depending on which gases are present in our set-up. Remember that certain gases will be excluded depending on which bands were chosen for the retrieval.

In [ ]:
# Calculate optimal estimation-related quantities..
q = RE.calculate_OE_quantities(solver);
h = RE.create_pressure_weights(buf.scene.atmosphere);
gas_co2 = RE.get_gas_from_name(buf.scene.atmosphere, "CO2");

idx = RE.idx_for_profile_sve(gas_co2, solver.state_vector);

In [ ]:
RE.calculate_xgas(buf.scene.atmosphere)

In [ ]:
# Calculate XCO2 uncert

In [ ]:
sqrt(h' * (q.Shat[idx, idx] * h)) # ucert in ppm

In [ ]:
# Do a correlation matrix based on posterior covariance
C = similar(q.Shat);
for idx in CartesianIndices(C)
    i, j = idx.I

    C[i,j] = q.Shat[i,j] / sqrt(q.Shat[i,i] * q.Shat[j,j])

end

Plots.heatmap(C, yflip=true, clims=(-1,1), c=:RdBu_5,
    rightmargin=20Plots.mm,
    colorbar_title=" \n\nCorrelation coefficient", colorbar_titlefont=(8, "JuliaMono-Regular"))
Plots.xlabel!("State vector element #")
Plots.ylabel!("State vector element #")

In [ ]:
#=
    Calculate and plot the XCO2 averaging kernel (normalized)
=#

# This grabs the indices within the state vector that correspond 
# to the CO2 profile.
idx = RE.idx_for_profile_sve(gas_co2, solver.state_vector)

if length(idx) > 0

    # Calculate the normalized averaging kernel
    # (remember, we calculated the pressure weights as `h` before)
    ak_norm = (h' * q.AK[idx,idx])' ./ h
    
    # Plot it!
    Plots.plot(
        ak_norm,
        buf.scene.atmosphere.pressure_levels,
        marker=:o,
        yflip=true,
        label=nothing,
        size=(400,400)
        )
    Plots.xlabel!("Normalized Averaging Kernel")
    Plots.ylabel!("Pressure level [$(buf.scene.atmosphere.pressure_unit)]")

else

    @warn "Sorry - cannot plot XCO2 AK"
    @warn "Probably no CO2 band retrieved.."

end